# Лабораторная работа по теме Итерационные методы решения слау

### Вариант 4
Метод наискорейшего градиентного спуска

##### описание метода
Метод наискорейшего градиентного спуска применяется для решения системы линейных алгебраических уравнений (СЛАУ) вида: $Ax=b$ \
где A - симметричная положительно определенная матрица n×n, x — вектор неизвестных, b — вектор правых частей.

In [25]:
import numpy as np
from typing import Tuple, Optional
import time

In [26]:


class GradientDescentSolver:
    """
    Реализация метода наискорейшего градиентного спуска
    для решения системы линейных уравнений Ax = b
    """
    
    def __init__(self, A: np.ndarray, b: np.ndarray):
        """
        Инициализация решателя
        
        Parameters:
        -----------
        A : np.ndarray
            Матрица коэффициентов (n x n)
        b : np.ndarray
            Вектор правых частей (n,)
        """
        self.A = A.astype(float)
        self.b = b.astype(float)
        self.n = A.shape[0]
        
        # Проверка размерностей
        if A.shape[0] != A.shape[1]:
            raise ValueError("Матрица A должна быть квадратной")
        if A.shape[0] != b.shape[0]:
            raise ValueError("Размеры A и b не совпадают")
            
        # Проверка симметричности и положительной определенности
        if not np.allclose(A, A.T):
            print("Предупреждение: Матрица A не симметрична!")
        # Попробуем проверить положительную определенность через собственные значения
        eigvals = np.linalg.eigvals(A)
        if np.any(eigvals <= 0):
            print("Предупреждение: Матрица A может не быть положительно определенной!")
    
    def solve(self, 
              x0: Optional[np.ndarray] = None, 
              tol: float = 1e-6, 
              max_iter: int = 1000,
              verbose: bool = False) -> Tuple[np.ndarray, dict]:
        """
        Решение системы методом наискорейшего градиентного спуска
        
        Parameters:
        -----------
        x0 : np.ndarray, optional
            Начальное приближение
        tol : float
            Точность решения
        max_iter : int
            Максимальное число итераций
        verbose : bool
            Флаг вывода информации о процессе
            
        Returns:
        --------
        x : np.ndarray
            Решение системы
        info : dict
            Информация о процессе решения
        """
        # Начальное приближение
        if x0 is None:
            x0 = np.zeros(self.n)
        else:
            x0 = x0.astype(float)
            
        x = x0.copy()
        r = self.A @ x - self.b  # Невязка
        
        # История для анализа сходимости
        residuals = [np.linalg.norm(r)]
        errors = []
        true_x = np.linalg.solve(self.A, self.b) if self.n < 1000 else None
        if true_x is not None:
            errors.append(np.linalg.norm(x - true_x))
        
        start_time = time.time()
        
        # Итерационный процесс
        for k in range(max_iter):
            # Вычисление оптимального шага
            Ar = self.A @ r
            alpha = np.dot(r, r) / np.dot(r, Ar)
            
            # Обновление решения
            x_new = x - alpha * r
            
            # Вычисление новой невязки
            r_new = self.A @ x_new - self.b
            
            # Сохранение истории
            residuals.append(np.linalg.norm(r_new))
            if true_x is not None:
                errors.append(np.linalg.norm(x_new - true_x))
            
            # Проверка критерия остановки
            if np.linalg.norm(r_new) < tol:
                x = x_new
                r = r_new
                if verbose:
                    print(f"Сходимость достигнута на итерации {k+1}")
                break
            
            # Обновление переменных для следующей итерации
            x = x_new
            r = r_new
            
            # Вывод информации
            if verbose and (k+1) % 100 == 0:
                print(f"Итерация {k+1}: норма невязки = {residuals[-1]:.6e}")
        
        else:
            if verbose:
                print(f"Достигнуто максимальное число итераций {max_iter}")
        
        end_time = time.time()
        
        # Сбор информации
        info = {
            'iterations': min(k + 1, max_iter),
            'residual': float(np.linalg.norm(r)),
            'residuals': residuals,
            'errors': errors if errors else None,
            'time': end_time - start_time,
            'converged': np.linalg.norm(r) < tol
        }
        
        return x, info
    

In [27]:
matrix = np.array([
    [5, 1, 1, 1],
    [1, 5, 1, 1],
    [1, 1, 5, 1],
    [1, 1, 1, 5],
])
d_array = np.array([
    1,
    1,
    3,
    3,
])

In [28]:
np_solving = np.linalg.solve(matrix, d_array)
np_solving

array([1.75785312e-17, 2.31296463e-17, 5.00000000e-01, 5.00000000e-01])

In [29]:
solver = GradientDescentSolver(matrix, d_array)
my_solving, info = solver.solve(tol=1e-10, max_iter=1000, verbose=True)
my_solving

Сходимость достигнута на итерации 19


array([9.32530846e-12, 9.32530846e-12, 5.00000000e-01, 5.00000000e-01])

In [30]:
max_error = 0
for np_x, my_x in zip(np_solving, my_solving):
    max_error = max(max_error, abs(np_x - my_x))
max_error

np.float64(9.32529088071704e-12)